In [98]:
import csv
import keras
import pyspark
import numpy as np
from pyspark.sql import SparkSession
from keras.preprocessing.text import Tokenizer
#from pyspark.ml.feature import NGram
import re
import cPickle as pickle
from pyspark.sql import SQLContext
from pyspark.sql import Row

In [102]:
sc.stop()
sc = pyspark.SparkContext(appName="preprocessor")
ss = SparkSession(sc)
sqlContext = SQLContext(sc)

In [103]:
def remove_urls(tweet):
    return re.sub(r"http\S+", "", tweet)

In [104]:
import ast

jdf = ss.read.json("../WebServer/sampledata_apple").rdd.map(list)
twitter_data = jdf.map(lambda l: l[1].encode("utf-8").replace("Status", ""))
# twitter_data = twitter_data.map(lambda l: ast.literal_eval(l))

In [105]:
# twitter_data = twitter_data.map(lambda l: re.findall('\((.*?)\)',l))
twitter_data.first()

'[(ID=938662696630865921, ScreenName=taetaeology, Created=Thu Dec 07 06:53:21 +0000 2017, Text=\'RT @btsanalytics: #CrystalSnow_BTS \xe2\x9d\x84\xef\xb8\x8f is also available to purchase on iTunes along with @BTS_twt JP \xf0\x9f\x87\xaf\xf0\x9f\x87\xb5 versions of MIC Drop and DNA \xe2\xa4\xb5\xef\xb8\x8f\\n\\n(h\xe2\x80\xa6\'), (ID=938662695888408576, ScreenName=ljok1999, Created=Thu Dec 07 06:53:20 +0000 2017, Text=\'The apple doent fall far from the tree! #CriminalDemoRATS https://t.co/Fup2dy9jFj\'), (ID=938662694072324097, ScreenName=bexx322, Created=Thu Dec 07 06:53:20 +0000 2017, Text=\'RT @bts_bighit: #ARMYs thank you for making #MICDropRemix the first K-Pop artist featured on the A List Pop playlist on Apple Music! https:\xe2\x80\xa6\'), (ID=938662686300151808, ScreenName=demetrafageria4, Created=Thu Dec 07 06:53:18 +0000 2017, Text=\'RT @Anggun_Cipta: Hi Singapore! Come and see me tomorrow, Friday Dec 8th at Apple Orchard Road for the launch of my new album #Anggun8 \xf0\x9f\x

In [106]:
def parse(s):
    tuples = s.split('), ')
    out = []
    for x in tuples:
        
        # print(x)
        # a, b, c, d = x.strip('(').split(',', 3).strip("(")
        a = x.strip("(").strip(")").split(',', 3)# x.strip("(").split(',', 3))  # ((int(a), str(b), str(c), str(d)))
    return a

# arr = twitter_data.first().replace("[", "").replace("]", "")
# print(parse(arr))
clean_data = twitter_data.map(lambda l: l.replace("[", "").replace("]", ""))
clean_data = clean_data.map(remove_urls).map(lambda l: re.sub(r"@\S+", "", l))
clean_data = clean_data.map(lambda l: re.sub(r"#\S+", "", l))
# print(clean_data.first())
# cleaner_data = clean_data.map(parse)
flat_data = clean_data.flatMap(lambda l: l.split("),"))
flat_data = flat_data.map(lambda l: l+")")
flat_data = flat_data.map(lambda l: l.replace("ID=", "").replace("ScreenName=","").replace("Created=","").replace("Text=",""))
flat_data = flat_data.map(lambda l: parse(str(l)))
flat_data.take(5)

[['938662696630865921',
  ' taetaeology',
  ' Thu Dec 07 06:53:21 +0000 2017',
  " 'RT   \xe2\x9d\x84\xef\xb8\x8f is also available to purchase on iTunes along with  JP \xf0\x9f\x87\xaf\xf0\x9f\x87\xb5 versions of MIC Drop and DNA \xe2\xa4\xb5\xef\xb8\x8f\\n\\n(h\xe2\x80\xa6'"],
 [' (938662695888408576',
  ' ljok1999',
  ' Thu Dec 07 06:53:20 +0000 2017',
  " 'The apple doent fall far from the tree!   (938662694072324097, bexx322, Thu Dec 07 06:53:20 +0000 2017, 'RT   thank you for making  the first K-Pop artist featured on the A List Pop playlist on Apple Music!  (938662686300151808, demetrafageria4, Thu Dec 07 06:53:18 +0000 2017, 'RT  Hi Singapore! Come and see me tomorrow, Friday Dec 8th at Apple Orchard Road for the launch of my new album  \xf0\x9f\x92\x9d I\xe2\x80\x99\xe2\x80\xa6'"],
 [' (938662686128246784',
  ' waiwtcast',
  ' Thu Dec 07 06:53:18 +0000 2017',
  " 'RT  We watched  a movie about how important it is that Postal Service workers get access to therapy as part of\xe2

In [97]:
'''
data_frame_schema = StructType([
    StructField("ID", IntType()),
    StructField("ScreenName", StringType()),
    StructField("Created", StringType()),
    StructField("Text", StringType())
])
'''
'''
dict_to_row = lambda d: Row(ID=d.get("ID"), ScreenName=d.get("ScreenName"), Created=d.get("Created"), Text=d.get("Text"))
'''
return_data = flat_data.map(lambda l: str(l[3]))
return_list = return_data.collect()

row_data = flat_data.map(lambda p: Row(
    ID=p[0], 
    ScreenName=str(p[1]),
    Created=str(p[2]),
    Text=str(p[3])
    )
)

# row_data.first()

In [86]:
twitter_df = sqlContext.createDataFrame(row_data)
twitter_df.registerTempTable("TwitterData")

In [87]:
twitter_df.show(3)

+--------------------+--------------------+------------+--------------------+
|             Created|                  ID|  ScreenName|                Text|
+--------------------+--------------------+------------+--------------------+
| Thu Dec 07 06:53...|  938662696630865921| taetaeology| 'RT   âï¸ is ...|
| Thu Dec 07 06:53...| (938662695888408576|    ljok1999| 'The apple doent...|
| Thu Dec 07 06:53...| (938662686128246784|   waiwtcast| 'RT  We watched ...|
+--------------------+--------------------+------------+--------------------+
only showing top 3 rows



In [116]:
namelist = ["taetaeology", "abc"]

new_df = twitter_df.where(twitter_df.ScreenName.isin(namelist))
# twit_df.show(10)
# sqlContext.sql("SELECT Text FROM TwitterData WHERE ScreenName = \"taetaeology\"").show(10)

In [117]:
new_df.show()

Py4JJavaError: An error occurred while calling o1988.showString.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2014)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2043)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2062)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:336)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollect(limit.scala:38)
	at org.apache.spark.sql.Dataset.org$apache$spark$sql$Dataset$$collectFromPlan(Dataset.scala:2853)
	at org.apache.spark.sql.Dataset$$anonfun$head$1.apply(Dataset.scala:2153)
	at org.apache.spark.sql.Dataset$$anonfun$head$1.apply(Dataset.scala:2153)
	at org.apache.spark.sql.Dataset$$anonfun$55.apply(Dataset.scala:2837)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:65)
	at org.apache.spark.sql.Dataset.withAction(Dataset.scala:2836)
	at org.apache.spark.sql.Dataset.he